In [70]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import requests
import io
import time
from collections import Counter
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score, classification_report

In [71]:
def load_dataset(csv_url):
  return pd.read_csv(csv_url)

general_data = load_dataset("https://raw.githubusercontent.com/NicolasDrapier/ProjetIA/main/general_data.csv")
employee_survey_data = load_dataset("https://raw.githubusercontent.com/NicolasDrapier/ProjetIA/main/employee_survey_data.csv")
manager_survey_data = load_dataset("https://raw.githubusercontent.com/NicolasDrapier/ProjetIA/main/manager_survey_data.csv")
in_data = load_dataset("https://raw.githubusercontent.com/NicolasDrapier/ProjetIA/main/in_time.csv")
out_data = load_dataset("https://raw.githubusercontent.com/NicolasDrapier/ProjetIA/main/out_time.csv")


In [72]:
data = pd.merge(general_data, employee_survey_data, on='EmployeeID')
data = pd.merge(data, manager_survey_data, on='EmployeeID')
data.set_index('EmployeeID', inplace=True)

In [73]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3
2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4
3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3
4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3
5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [74]:
data['NumCompaniesWorked'].fillna(0, inplace=True)
data['JobSatisfaction'].fillna(data['JobSatisfaction'].median(), inplace=True)
data['EnvironmentSatisfaction'].fillna(data['EnvironmentSatisfaction'].median(), inplace=True)
data['WorkLifeBalance'].fillna(data['WorkLifeBalance'].median(), inplace=True)

for index, row in data[data['TotalWorkingYears'].isna()].iterrows():
  data.at[index, 'TotalWorkingYears'] = data.loc[data['Age'] == row['Age']]['TotalWorkingYears'].median()

In [75]:
data.drop(['StandardHours', 'EmployeeCount', 'Over18'], axis=1, inplace=True)

In [76]:
data.replace({'No': 0, 'Yes':1}, inplace=True)

In [77]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,
1,51,0,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3
2,31,1,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4
3,32,0,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,Married,193280,1.0,15,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3
4,38,0,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,Married,83210,3.0,11,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3
5,32,0,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,Single,23420,4.0,12,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [78]:
le = LabelEncoder()
categories = ['BusinessTravel', 'Gender']
for c in categories:
  data[c] = le.fit_transform(data[c])

In [79]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,
1,51,0,2,Sales,6,2,Life Sciences,0,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3
2,31,1,1,Research & Development,10,1,Life Sciences,0,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4
3,32,0,1,Research & Development,17,4,Other,1,4,Sales Executive,Married,193280,1.0,15,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3
4,38,0,0,Research & Development,2,5,Life Sciences,1,3,Human Resources,Married,83210,3.0,11,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3
5,32,0,2,Research & Development,10,1,Medical,1,1,Sales Executive,Single,23420,4.0,12,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [80]:
in_data.dropna(axis=1, how='all', inplace=True)
out_data.dropna(axis=1, how='all', inplace=True)
in_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
out_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
in_data.set_index("EmployeeID", inplace=True)
out_data.set_index("EmployeeID", inplace=True)

In [81]:
for col in in_data.columns:
  in_data[col] = pd.to_datetime(in_data[col], errors='coerce', format='%Y-%m-%d %H:%M:%S')
  out_data[col] = pd.to_datetime(out_data[col], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [82]:
t = out_data-in_data
b = t.mean(axis=1)
m = pd.DataFrame(b, columns=["MeanTimeAtWorkByYear"])
m

,MeanTimeAtWorkByYear
EmployeeID,
1,0 days 07:22:25.142241379
2,0 days 07:43:08.288135593
3,0 days 07:00:47.665289256
4,0 days 07:11:37.242553191
5,0 days 08:00:22.228571428
...,...
4406,0 days 08:31:20.197530864
4407,0 days 06:05:34.634854771
4408,0 days 07:42:23.874458874


In [83]:
l = [row['MeanTimeAtWorkByYear'].total_seconds() for i, row in m.iterrows()]
l = [x/3600 for x in l]
m['MeanTimeAtWorkByYear'] = l
m

,MeanTimeAtWorkByYear
EmployeeID,
1,7.373651
2,7.718969
3,7.013240
4,7.193678
5,8.006175
...,...
4406,8.522277
4407,6.092954
4408,7.706632


In [84]:
encoded = pd.get_dummies(data.copy(), ['Department', 'EducationField', 'JobRole', 'MaritalStatus'])
encoded.head()

,Age,Attrition,BusinessTravel,DistanceFromHome,Education,Gender,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,51,0,2,6,2,0,1,131160,1.0,11,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,31,1,1,10,1,0,1,41890,0.0,23,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,32,0,1,17,4,1,4,193280,1.0,15,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
4,38,0,0,2,5,1,3,83210,3.0,11,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,32,0,2,10,1,1,1,23420,4.0,12,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [86]:
encoded = pd.merge(encoded, m, on="EmployeeID")
encoded.shape

(4410, 43)

In [87]:
encoded

,Age,Attrition,BusinessTravel,DistanceFromHome,Education,Gender,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MeanTimeAtWorkByYear
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,51,0,2,6,2,0,1,131160,1.0,11,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,7.373651
2,31,1,1,10,1,0,1,41890,0.0,23,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,7.718969
3,32,0,1,17,4,1,4,193280,1.0,15,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7.013240
4,38,0,0,2,5,1,3,83210,3.0,11,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,7.193678
5,32,0,2,10,1,1,1,23420,4.0,12,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,8.006175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,42,0,2,5,4,0,1,60290,3.0,17,1,10.0,5,3,0,2,4.0,1.0,3.0,3,3,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,8.522277
4407,29,0,2,2,4,1,1,26790,2.0,15,0,10.0,2,3,0,2,4.0,4.0,3.0,2,3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,6.092954
4408,25,0,2,25,2,1,2,37020,0.0,20,0,5.0,4,4,1,2,1.0,3.0,3.0,3,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,7.706632


In [88]:
encoded.to_csv('./mix_encoded.csv')